In [33]:
import pandas as pd

In [ ]:
# Charger le fichier GTDB 220 (ajuste le chemin si nécessaire)
gtdb_220 = pd.read_csv(
    "/shared/projects/mudis4ls_is4_benchmark/gtdb/bac120_taxonomy.tsv.xz", sep="\t", header=None, names=["genome_id", "taxonomy"]
    )

# Extraire les genres (niveau "g__")
gtdb_220["genus"] = gtdb_220["taxonomy"].str.extract(r";g__([^;]+)")

# Supprimer les NaN et créer un ensemble pour un accès rapide
known_genera = set(gtdb_220["genus"].dropna().unique())

# Vérification : Afficher les 10 premiers genres
print(list(known_genera)[:10])

In [ ]:
def extract_genus_species(species_name):
    words = species_name.split()
    
    # Cas où le premier mot est un genre valide
    if words[0] in known_genera:
        genus = words[0]
        species = " ".join(words[1:]) if len(words) > 1 else "sp."
    else:
        genus = None  # Pas un genre connu → classer en inconnu
        species = species_name  # Garder la classification entière
    
    return genus, species

# Appliquer sur la colonne "species" du dataframe Bracken
bracken_output[["genus", "species"]] = bracken_output["species"].apply(
    lambda x: pd.Series(extract_genus_species(x))
)
